In [269]:
import torch
import data_loader
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue as prologue

In [270]:
from torch.utils.data import TensorDataset, DataLoader

def load_data(N=1000, batch_size=50, seed=42):
    # Load data
    train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)
    train_target = torch.nn.functional.one_hot(train_target)
    test_target = torch.nn.functional.one_hot(test_target)
    
    train_input = train_input.to(device)
    train_target = train_target.to(device)
    train_classes = train_classes.to(device)
    test_input = test_input.to(device)
    test_target = test_target.to(device)
    test_classes = test_classes.to(device)
    # Normalize data
    mean, std = train_input.mean(), train_input.std()
    train_input.sub_(mean).div_(std)
    test_input.sub_(mean).div_(std)
    
    # Generate dataset
    train_data = TensorDataset(train_input, train_target, train_classes)
    test_data = TensorDataset(test_input, test_target, test_classes)
    
    # For reproducibility
    torch.manual_seed(seed)
    
    # Generate data loader
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    return train_loader, test_loader

In [271]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [272]:
# helper.py

# Count the number of parameters
def count_param(model):
    return sum([torch.numel(param) for param in model.parameters()])

In [273]:
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=42)

In [274]:
def compute_nb_errors(model, data_loader):

    nb_data_errors = 0

    for data_input, data_target, data_classes in data_loader:
        output = model(data_input)
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [275]:
def compute_nb_errors_siamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [276]:
def compute_nb_errors_auxsiamese(model, data_loader):

    nb_data_errors = 0
    for data_input, data_target, data_classes in data_loader:
        data_1, data_2 = data_input.unbind(1)               
        output, aux1, aux2 = model(data_1.unsqueeze(1), data_2.unsqueeze(1))
        nb_error = torch.sum(torch.argmax(output, dim=1, keepdim=True) != torch.argmax(data_target, dim=1, keepdim=True))
        nb_data_errors += nb_error
        
    return nb_data_errors

In [277]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(392, 160)
        self.fc2 = nn.Linear(160, 64) 
        self.fc3 = nn.Linear(64, 2) 


    def forward(self, x):
        x = x.view(-1,392) # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x

In [278]:
class AuxMLP(nn.Module):
    def __init__(self):
        super(AuxMLP, self).__init__()
        
        self.fc11 = nn.Linear(196, 160)
        self.fc12 = nn.Linear(196, 160)
        self.fc21 = nn.Linear(160, 10)
        self.fc22 = nn.Linear(160, 10)
        self.fc3 = nn.Linear(20, 2) 


    def forward(self, x1, x2):
        x1 = x1.view(-1,196) # flatten
        x1 = F.relu(self.fc11(x1))
        x1 = self.fc21(x1)
        aux1 = F.softmax(x1)
        x1 = F.relu(x1)
        
        x2 = x2.view(-1,196) # flatten
        x2 = F.relu(self.fc12(x2))
        x2 = self.fc22(x2)
        aux2 = F.softmax(x2)
        x2 = F.relu(x2)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc3(x))

        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        return x, aux1, aux2

In [279]:
class SiameseMLP(nn.Module):
    def __init__(self):
        super(SiameseMLP, self).__init__()
        
        self.fc1 = nn.Linear(196, 160)
        self.fc2 = nn.Linear(160, 10)
        self.fc3 = nn.Linear(20, 2) 


    def forward(self, x1, x2):
        x1 = x1.view(-1,196) # flatten
        x1 = F.relu(self.fc1(x1))
        x1 = self.fc2(x1)
        x1 = F.relu(x1)
        
        x2 = x2.view(-1,196) # flatten
        x2 = F.relu(self.fc1(x2))
        x2 = self.fc2(x2)
        x2 = F.relu(x2)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc3(x))

        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        return x

In [280]:
class AuxsiameseMLP(nn.Module):
    def __init__(self):
        super(AuxsiameseMLP, self).__init__()
        
        self.fc1 = nn.Linear(196, 160)
        self.fc2 = nn.Linear(160, 10)
        self.fc3 = nn.Linear(20, 2) 


    def forward(self, x1, x2):
        x1 = x1.view(-1,196) # flatten
        x1 = F.relu(self.fc1(x1))
        x1 = self.fc2(x1)
        aux1 = F.softmax(x1)
        x1 = F.relu(x1)
        
        x2 = x2.view(-1,196) # flatten
        x2 = F.relu(self.fc1(x2))
        x2 = self.fc2(x2)
        aux2 = F.softmax(x2)
        x2 = F.relu(x2)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc3(x))

        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        return x, aux1, aux2

In [281]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size=3)    # size [nb, 32, 12, 12]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(10, 2)
        
    def forward(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 6, 6]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        x = x.view(-1, 256) # size [nb, 256]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [282]:
class SiameseBaseNet(nn.Module):
    def __init__(self):
        super(SiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = F.relu(self.fc2(x1))
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = F.relu(self.fc2(x2))
        
        x = torch.cat([x1, x2], dim=1)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x

In [283]:
class AuxsiameseBaseNet(nn.Module):
    def __init__(self):
        super(AuxsiameseBaseNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = self.convs(x1)
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc1(x1)))
        x1 = self.fc2(x1)
        aux1 = F.softmax(x1)
        x1 = F.relu(x1)
        
        x2 = self.convs(x2)
        x2 = x2.view(-1, 256)
        x2 = F.relu(self.fc1(x2))
        x2 = self.fc2(x2)
        aux2 = F.softmax(x2)
        x2 = F.relu(x2)
        
        x = torch.cat([x1, x2], dim=1)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [284]:
class AuxBaseNet(nn.Module):
    def __init__(self):
        super(AuxBaseNet, self).__init__()
        self.conv11 = nn.Conv2d(1, 32, kernel_size=3)    # size [nb, 32, 10, 10]
        self.conv21 = nn.Conv2d(32, 64, kernel_size=3)   # size [nb, 64, 4, 4]
        self.fc11 = nn.Linear(256, 200)
        self.fc21 = nn.Linear(200, 10)
        self.conv12 = nn.Conv2d(1, 32, kernel_size=5)    # size [nb, 32, 10, 10]
        self.conv22 = nn.Conv2d(32, 64, kernel_size=2)   # size [nb, 64, 4, 4]
        self.fc12 = nn.Linear(256, 200)
        self.fc22 = nn.Linear(200, 10)
        self.fc3 = nn.Linear(20, 2)
        
    def convs(self, x):        
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2)) # size [nb, 32, 5, 5]      
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) # size [nb, 64, 2, 2]
        return x
    
    def forward(self, x1, x2):
        x1 = F.relu(F.max_pool2d(self.conv11(x1), kernel_size=2)) # size [nb, 32, 5, 5]  
        x1 = F.relu(F.max_pool2d(self.conv21(x1), kernel_size=2)) # size [nb, 64, 2, 2]
        x1 = x1.view(-1, 256)
        x1 = F.relu((self.fc11(x1)))
        x1 = self.fc21(x1)
        aux1 = F.softmax(x1)
        x1 = F.relu(x1)
        
        x2 = F.relu(F.max_pool2d(self.conv12(x2), kernel_size=2)) # size [nb, 32, 5, 5]  
        x2 = F.relu(F.max_pool2d(self.conv22(x2), kernel_size=2)) # size [nb, 64, 2, 2]
        x2 = x2.view(-1, 256)
        x2 = F.relu((self.fc12(x2)))
        x2 = self.fc22(x2)
        aux2 = F.softmax(x2)
        x2 = F.relu(x2)
        
        x = torch.cat([x1, x2], dim=1)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))
        
        x = torch.sigmoid(self.fc3(x))
        
        return x, aux1, aux2

In [285]:
# ResNetBlock with skip-connection and batch normalization
class ResNetBlock(nn.Module):
    def __init__(self, nb_channels, kernel_size, dropout = 0):
        super().__init__()

        self.conv1 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn1 = nn.BatchNorm2d(nb_channels)

        self.conv2 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn2 = nn.BatchNorm2d(nb_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        y = self.conv1(x)
        y = self.bn1(y)
        y = self.dropout(y)
        y = F.relu(y)
        y = self.conv2(y)
        y = self.bn2(y)
        y = self.dropout(y)
        y = y + x
        y = F.relu(y)

        return y

In [286]:
class ResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(288, nb_classes)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x = self.resnet_blocks(x)
        x = F.avg_pool2d(x, 4).view(x.size(0), -1)
        x = torch.sigmoid(self.fc(x))
        return x

In [287]:
class SiameseResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(576, nb_classes)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn(self.conv(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn(self.conv(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        #aux1 = F.softmax(self.fc1(x1)
        #aux2 = F.softmax(self.fc1(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        x = torch.sigmoid(self.fc(x))
        
        return x

In [288]:
class AuxResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv1 = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.conv2 = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn1 = nn.BatchNorm2d(nb_channels)
        self.bn2 = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks1 = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )
        self.resnet_blocks2 = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )
        
        self.fc = nn.Linear(576, nb_classes)
        self.fc1 = nn.Linear(288, 10)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn1(self.conv1(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks1(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn2(self.conv2(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks2(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc(x))
        aux1 = F.softmax(self.fc1(x1))
        aux2 = F.softmax(self.fc1(x2))
        #aux1 = F.softmax(x1)
        #aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        
        return x, aux1, aux2

In [289]:
class AuxsiameseResNet(nn.Module):

    def __init__(self, nb_residual_blocks, input_channels, nb_channels,
                 kernel_size = 3, nb_classes = 10, dropout = 0):
        super().__init__()

        self.conv = nn.Conv2d(input_channels, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size, dropout)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(576, nb_classes)
        self.fc1 = nn.Linear(288, 10)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, x1, x2):
        x1 = F.relu(self.bn(self.conv(x1)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x1 = self.resnet_blocks(x1)
        x1 = F.avg_pool2d(x1, 4).view(x1.size(0), -1)
        
        x2 = F.relu(self.bn(self.conv(x2)))
        #x = F.relu(self.dropout(self.bn(self.conv(x))))
        x2 = self.resnet_blocks(x2)
        x2 = F.avg_pool2d(x2, 4).view(x2.size(0), -1)
        
        x = torch.cat([x1, x2], dim=1)
        x = torch.sigmoid(self.fc(x))
        aux1 = F.softmax(self.fc1(x1))
        aux2 = F.softmax(self.fc1(x2))
        #aux1 = F.softmax(x1)
        #aux2 = F.softmax(x2)
        #x = torch.abs(x1 - x2)
        #x = F.relu(self.fc1(x.flatten(start_dim=1)))
        #x = F.relu(self.fc2(x))      
        
        return x, aux1, aux2

In [291]:
model = AuxsiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

78924

In [294]:
model = SiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

76034

In [296]:
model = AuxResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)

# Calculate the number of parameters in the model
count_param(model)

153804

In [297]:
model = ResNet(nb_residual_blocks = 4, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
# Calculate the number of parameters in the model
count_param(model)

75746

In [298]:
model = BaseNet()

# Calculate the number of parameters in the model
count_param(model)

72536

In [299]:
model = SiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

72268

In [300]:
model = AuxsiameseBaseNet()

# Calculate the number of parameters in the model
count_param(model)

72268

In [301]:
model = AuxBaseNet()

# Calculate the number of parameters in the model
count_param(model)

134766

In [302]:
def train(model, train_loader, eta, decay, n_epochs=25, verbose=False, siamese=False, aux=False, alpha = 0):

    #binary_crit = nn.CrossEntropyLoss()
    binary_crit = torch.nn.BCELoss()
    aux_crit = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=eta, weight_decay=decay)
    #optimizer = torch.optim.LBFGS(model.parameters(), lr=eta)
    tr_losses = []
    tr_accuracies = []

    for e in range(n_epochs):
        # Reset training/validation loss
        tr_loss = 0

        # Training model
        model.train()

        for train_input, train_target, train_classes in iter(train_loader):
            #train_target = torch.nn.functional.one_hot(train_target)
            # Forward pass
            
            if siamese == True:
                train_1, train_2 = train_input.unbind(1)
                if aux == True:
                    output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
                else:
                    output = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            elif aux == True:
                train_1, train_2 = train_input.unbind(1)
                output, aux1, aux2 = model(train_1.unsqueeze(1), train_2.unsqueeze(1))
            else:
                output = model(train_input)
                
            # Binary classification loss
            binary_loss = binary_crit(output, train_target.float())
            total_loss = binary_loss
            
            # Auxiliary loss
            if aux == True:

                aux_loss1 = aux_crit(aux1, train_classes[:,0])
                aux_loss2 = aux_crit(aux2, train_classes[:,1])
                aux_loss = aux_loss1 + aux_loss2
                total_loss = binary_loss + aux_loss * alpha
        
            # Total loss = Binary loss + aux loss * alpha
            
            tr_loss += total_loss

            # Backward pass
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

        # Collect accuracy data
        # tr_accuracies.append(compute_nb_errors_siamese(model, train_loader)/1000)

        # Collect loss data
        tr_losses.append(tr_loss)

        if verbose:
            print('Epoch %d/%d, Binary loss: %.3f' %
                  (e+1, n_epochs, tr_loss))

In [303]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = AuxsiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
model.to(device)
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True, aux=True, alpha = 1)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 96.767
Epoch 2/25, Binary loss: 83.466
Epoch 3/25, Binary loss: 77.176
Epoch 4/25, Binary loss: 71.319
Epoch 5/25, Binary loss: 69.567
Epoch 6/25, Binary loss: 68.520
Epoch 7/25, Binary loss: 67.732
Epoch 8/25, Binary loss: 67.278
Epoch 9/25, Binary loss: 66.580
Epoch 10/25, Binary loss: 65.992
Epoch 11/25, Binary loss: 65.673
Epoch 12/25, Binary loss: 65.069
Epoch 13/25, Binary loss: 64.851
Epoch 14/25, Binary loss: 64.775
Epoch 15/25, Binary loss: 63.923
Epoch 16/25, Binary loss: 63.595
Epoch 17/25, Binary loss: 63.249
Epoch 18/25, Binary loss: 62.958
Epoch 19/25, Binary loss: 62.756
Epoch 20/25, Binary loss: 62.658
Epoch 21/25, Binary loss: 62.580
Epoch 22/25, Binary loss: 62.476
Epoch 23/25, Binary loss: 62.411
Epoch 24/25, Binary loss: 62.374
Epoch 25/25, Binary loss: 62.351
Spend 5.805825e+01 s
tensor(1., device='cuda:0') tensor(0.8810, device='cuda:0')


In [84]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxsiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
model.to(device)
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 40.103
Epoch 2/25, Binary loss: 31.625
Epoch 3/25, Binary loss: 28.229
Epoch 4/25, Binary loss: 26.540
Epoch 5/25, Binary loss: 24.686
Epoch 6/25, Binary loss: 23.148
Epoch 7/25, Binary loss: 22.162
Epoch 8/25, Binary loss: 21.673
Epoch 9/25, Binary loss: 20.688
Epoch 10/25, Binary loss: 19.981
Epoch 11/25, Binary loss: 19.738
Epoch 12/25, Binary loss: 19.896
Epoch 13/25, Binary loss: 19.436
Epoch 14/25, Binary loss: 19.325
Epoch 15/25, Binary loss: 19.050
Epoch 16/25, Binary loss: 18.952
Epoch 17/25, Binary loss: 18.856
Epoch 18/25, Binary loss: 18.774
Epoch 19/25, Binary loss: 18.752
Epoch 20/25, Binary loss: 17.955
Epoch 21/25, Binary loss: 17.716
Epoch 22/25, Binary loss: 17.694
Epoch 23/25, Binary loss: 17.623
Epoch 24/25, Binary loss: 17.593
Epoch 25/25, Binary loss: 17.584
Spend 1.226981e+02 s
tensor(1., device='cuda:0') tensor(0.8610, device='cuda:0')


In [1081]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = SiameseResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 11.641
Epoch 2/25, Binary loss: 8.032
Epoch 3/25, Binary loss: 5.910
Epoch 4/25, Binary loss: 5.168
Epoch 5/25, Binary loss: 4.474
Epoch 6/25, Binary loss: 3.215
Epoch 7/25, Binary loss: 3.069
Epoch 8/25, Binary loss: 3.053
Epoch 9/25, Binary loss: 2.638
Epoch 10/25, Binary loss: 2.044
Epoch 11/25, Binary loss: 2.564
Epoch 12/25, Binary loss: 1.829
Epoch 13/25, Binary loss: 1.344
Epoch 14/25, Binary loss: 0.564
Epoch 15/25, Binary loss: 0.404
Epoch 16/25, Binary loss: 0.158
Epoch 17/25, Binary loss: 0.359
Epoch 18/25, Binary loss: 0.458
Epoch 19/25, Binary loss: 0.738
Epoch 20/25, Binary loss: 0.517
Epoch 21/25, Binary loss: 0.761
Epoch 22/25, Binary loss: 0.393
Epoch 23/25, Binary loss: 0.200
Epoch 24/25, Binary loss: 0.130
Epoch 25/25, Binary loss: 0.067
Spend 4.333027e+02 s
tensor(0.9990) tensor(0.8690)


In [164]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
model = ResNet(nb_residual_blocks = 4, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
model.to(device)
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 12.063
Epoch 2/25, Binary loss: 8.823
Epoch 3/25, Binary loss: 6.645
Epoch 4/25, Binary loss: 5.994
Epoch 5/25, Binary loss: 4.474
Epoch 6/25, Binary loss: 3.104
Epoch 7/25, Binary loss: 2.493
Epoch 8/25, Binary loss: 1.249
Epoch 9/25, Binary loss: 0.706
Epoch 10/25, Binary loss: 0.315
Epoch 11/25, Binary loss: 0.129
Epoch 12/25, Binary loss: 0.076
Epoch 13/25, Binary loss: 0.047
Epoch 14/25, Binary loss: 0.038
Epoch 15/25, Binary loss: 0.030
Epoch 16/25, Binary loss: 0.029
Epoch 17/25, Binary loss: 0.023
Epoch 18/25, Binary loss: 0.019
Epoch 19/25, Binary loss: 0.017
Epoch 20/25, Binary loss: 0.016
Epoch 21/25, Binary loss: 0.015
Epoch 22/25, Binary loss: 0.012
Epoch 23/25, Binary loss: 0.011
Epoch 24/25, Binary loss: 0.011
Epoch 25/25, Binary loss: 0.011
Spend 3.890840e+01 s
tensor(1., device='cuda:0') tensor(0.8460, device='cuda:0')


In [878]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
model = SiameseBaseNet()
#model = BaseNet()
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 2.743
Epoch 2/25, Binary loss: 2.704
Epoch 3/25, Binary loss: 2.636
Epoch 4/25, Binary loss: 2.529
Epoch 5/25, Binary loss: 2.372
Epoch 6/25, Binary loss: 2.187
Epoch 7/25, Binary loss: 1.991
Epoch 8/25, Binary loss: 1.832
Epoch 9/25, Binary loss: 1.687
Epoch 10/25, Binary loss: 1.552
Epoch 11/25, Binary loss: 1.421
Epoch 12/25, Binary loss: 1.303
Epoch 13/25, Binary loss: 1.189
Epoch 14/25, Binary loss: 1.096
Epoch 15/25, Binary loss: 1.001
Epoch 16/25, Binary loss: 0.925
Epoch 17/25, Binary loss: 0.836
Epoch 18/25, Binary loss: 0.760
Epoch 19/25, Binary loss: 0.696
Epoch 20/25, Binary loss: 0.614
Epoch 21/25, Binary loss: 0.548
Epoch 22/25, Binary loss: 0.487
Epoch 23/25, Binary loss: 0.422
Epoch 24/25, Binary loss: 0.361
Epoch 25/25, Binary loss: 0.307
Spend 9.664215e+00 s
tensor(0.9890) tensor(0.8580)


In [268]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#modeF.avg_pool2dl = SiameseBaseNet()
model = BaseNet()
model.to(device)
train(model, train_loader, 0.001, 100, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 13.981
Epoch 2/25, Binary loss: 13.983
Epoch 3/25, Binary loss: 13.972
Epoch 4/25, Binary loss: 13.956
Epoch 5/25, Binary loss: 13.943
Epoch 6/25, Binary loss: 13.928
Epoch 7/25, Binary loss: 13.918
Epoch 8/25, Binary loss: 13.908
Epoch 9/25, Binary loss: 13.899
Epoch 10/25, Binary loss: 13.892
Epoch 11/25, Binary loss: 13.886
Epoch 12/25, Binary loss: 13.881
Epoch 13/25, Binary loss: 13.877
Epoch 14/25, Binary loss: 13.873
Epoch 15/25, Binary loss: 13.871
Epoch 16/25, Binary loss: 13.869
Epoch 17/25, Binary loss: 13.867
Epoch 18/25, Binary loss: 13.866
Epoch 19/25, Binary loss: 13.865
Epoch 20/25, Binary loss: 13.864
Epoch 21/25, Binary loss: 13.864
Epoch 22/25, Binary loss: 13.863
Epoch 23/25, Binary loss: 13.863
Epoch 24/25, Binary loss: 13.863
Epoch 25/25, Binary loss: 13.863
Spend 1.280331e+01 s
tensor(0.5720, device='cuda:0') tensor(0.5230, device='cuda:0')


In [267]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#modeF.avg_pool2dl = SiameseBaseNet()
model = MLP()
model.to(device)
train(model, train_loader, 0.001, 25, verbose=True, siamese=False)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

Epoch 1/25, Binary loss: 13.847
Epoch 2/25, Binary loss: 13.813
Epoch 3/25, Binary loss: 13.823
Epoch 4/25, Binary loss: 13.830
Epoch 5/25, Binary loss: 13.837
Epoch 6/25, Binary loss: 13.843
Epoch 7/25, Binary loss: 13.849
Epoch 8/25, Binary loss: 13.853
Epoch 9/25, Binary loss: 13.855
Epoch 10/25, Binary loss: 13.858
Epoch 11/25, Binary loss: 13.859
Epoch 12/25, Binary loss: 13.860
Epoch 13/25, Binary loss: 13.860
Epoch 14/25, Binary loss: 13.861
Epoch 15/25, Binary loss: 13.861
Epoch 16/25, Binary loss: 13.861
Epoch 17/25, Binary loss: 13.861
Epoch 18/25, Binary loss: 13.861
Epoch 19/25, Binary loss: 13.861
Epoch 20/25, Binary loss: 13.861
Epoch 21/25, Binary loss: 13.861
Epoch 22/25, Binary loss: 13.861
Epoch 23/25, Binary loss: 13.862
Epoch 24/25, Binary loss: 13.861
Epoch 25/25, Binary loss: 13.861
Spend 8.013068e+00 s
tensor(0.5650, device='cuda:0') tensor(0.5550, device='cuda:0')


In [184]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxsiameseBaseNet()
model.to(device)
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=True, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 40.692
Epoch 2/25, Binary loss: 36.508
Epoch 3/25, Binary loss: 33.839
Epoch 4/25, Binary loss: 32.298
Epoch 5/25, Binary loss: 31.083
Epoch 6/25, Binary loss: 30.109
Epoch 7/25, Binary loss: 29.243
Epoch 8/25, Binary loss: 28.548
Epoch 9/25, Binary loss: 27.797
Epoch 10/25, Binary loss: 27.353
Epoch 11/25, Binary loss: 26.681
Epoch 12/25, Binary loss: 26.124
Epoch 13/25, Binary loss: 25.867
Epoch 14/25, Binary loss: 25.536
Epoch 15/25, Binary loss: 25.161
Epoch 16/25, Binary loss: 25.083
Epoch 17/25, Binary loss: 25.024
Epoch 18/25, Binary loss: 24.913
Epoch 19/25, Binary loss: 24.722
Epoch 20/25, Binary loss: 24.646
Epoch 21/25, Binary loss: 24.586
Epoch 22/25, Binary loss: 24.558
Epoch 23/25, Binary loss: 24.581
Epoch 24/25, Binary loss: 24.514
Epoch 25/25, Binary loss: 24.502
Spend 1.869615e+01 s
tensor(1., device='cuda:0') tensor(0.8450, device='cuda:0')


In [175]:
import time

number = torch.randint(1,50,(1,))
train_loader, test_loader = load_data(N=1000, batch_size=50, seed=number)
time1 = time.perf_counter()
#model = SiameseBaseNet()
#model = BaseNet()
model = AuxBaseNet()
model.to(device)
train(model, train_loader, 0.001, 0, 25, verbose=True, siamese=False, aux=True, alpha = 0.3)
time2 = time.perf_counter()
print('Spend {:e} s'.format(time2 - time1))

tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
print(tr_accuracy, te_accuracy)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1/25, Binary loss: 41.120
Epoch 2/25, Binary loss: 38.946
Epoch 3/25, Binary loss: 36.412
Epoch 4/25, Binary loss: 34.846
Epoch 5/25, Binary loss: 33.540
Epoch 6/25, Binary loss: 32.230
Epoch 7/25, Binary loss: 30.754
Epoch 8/25, Binary loss: 29.528
Epoch 9/25, Binary loss: 28.194
Epoch 10/25, Binary loss: 26.966
Epoch 11/25, Binary loss: 25.878
Epoch 12/25, Binary loss: 25.103
Epoch 13/25, Binary loss: 24.781
Epoch 14/25, Binary loss: 23.868
Epoch 15/25, Binary loss: 23.551
Epoch 16/25, Binary loss: 23.321
Epoch 17/25, Binary loss: 22.649
Epoch 18/25, Binary loss: 22.447
Epoch 19/25, Binary loss: 21.881
Epoch 20/25, Binary loss: 21.647
Epoch 21/25, Binary loss: 21.554
Epoch 22/25, Binary loss: 21.414
Epoch 23/25, Binary loss: 21.244
Epoch 24/25, Binary loss: 21.043
Epoch 25/25, Binary loss: 20.986
Spend 2.300434e+01 s
tensor(1., device='cuda:0') tensor(0.8680, device='cuda:0')


In [304]:
accuracies8 = []
times8 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=32, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    model = MLP()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times8.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies8.append(te_accuracy)

print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies8).mean(), torch.tensor(accuracies8).std()))

tensor(1., device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8090, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8190, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7850, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8160, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7910, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8030, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8140, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7930, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7890, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8050, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8040, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8190, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8090, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.808

In [258]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times8).mean(), torch.tensor(times8).std()))

Mean: 12.561, Std: 0.105


In [196]:
accuracies9 = []
times9 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseMLP()
    model.to(device)
    #model = BaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times9.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies9.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies9).mean(), torch.tensor(accuracies9).std()))

tensor(0.9990, device='cuda:0') tensor(0.8300, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8080, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8040, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8250, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8110, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8420, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8220, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8230, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8340, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9970, device='cuda:0') tensor(0.8340, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8340, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8390, device='cuda:0')


In [251]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times9).mean(), torch.tensor(times9).std()))

Mean: 9.540, Std: 0.115


In [248]:
accuracies10 = []
times10 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxMLP()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.6)
    time2 = time.perf_counter()
    times10.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies10.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies10).mean(), torch.tensor(accuracies10).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9810, device='cuda:0') tensor(0.7970, device='cuda:0')
tensor(0.9700, device='cuda:0') tensor(0.8010, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(0.9930, device='cuda:0') tensor(0.8210, device='cuda:0')
tensor(0.9850, device='cuda:0') tensor(0.8190, device='cuda:0')
tensor(0.9850, device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8220, device='cuda:0')
tensor(0.9900, device='cuda:0') tensor(0.8230, device='cuda:0')
tensor(0.9810, device='cuda:0') tensor(0.8100, device='cuda:0')
tensor(0.9910, device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(0.9870, device='cuda:0') tensor(0.8270, device='cuda:0')
tensor(0.9850, device='cuda:0') tensor(0.8170, device='cuda:0')
tensor(0.9620, device='cuda:0') tensor(0.8140, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(0.9760, device='cuda:0') tensor(0.8090, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0

In [252]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times10).mean(), torch.tensor(times10).std()))

Mean: 13.360, Std: 0.120


In [249]:
accuracies11 = []
times11 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseMLP()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.5)
    time2 = time.perf_counter()
    times11.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies11.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies11).mean(), torch.tensor(accuracies11).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9740, device='cuda:0') tensor(0.8540, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8210, device='cuda:0')
tensor(0.9830, device='cuda:0') tensor(0.8740, device='cuda:0')
tensor(0.9780, device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9690, device='cuda:0') tensor(0.8430, device='cuda:0')
tensor(0.9840, device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9680, device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9820, device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(0.9750, device='cuda:0') tensor(0.8490, device='cuda:0')
tensor(0.9640, device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(0.9810, device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(0.9960, device='cuda:0') tensor(0.8570, device='cuda:0')
tensor(0.9910, device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(0.9520, device='cuda:0') tensor(0.8390, device='cuda:0')
tensor(0.9730, device='cuda:0') tensor(0.8470, device='cuda:0')
tensor(0.9820, device='cuda:0') tensor(0

In [253]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times11).mean(), torch.tensor(times11).std()))

Mean: 11.084, Std: 0.091


In [314]:
accuracies = []
times = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=8, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    model = BaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times12.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies.append(te_accuracy)

print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies).mean(), torch.tensor(accuracies).std()))

tensor(1., device='cuda:0') tensor(0.8170, device='cuda:0')
tensor(0.9740, device='cuda:0') tensor(0.7800, device='cuda:0')
tensor(0.5450, device='cuda:0') tensor(0.5580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(0.9720, device='cuda:0') tensor(0.7980, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8410, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8120, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8180, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8300, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8310, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8150, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8030, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8190, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8200, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.7960, device='cuda:0')
tensor(1., device='cuda:0') 

In [147]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times).mean(), torch.tensor(times).std()))

Mean: 12.519, Std: 0.134


In [79]:
accuracies1 = []
times1 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseBaseNet()
    model.to(device)
    #model = BaseNet()
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times1.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies1.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies1).mean(), torch.tensor(accuracies1).std()))

tensor(1., device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8390, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8710, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8570, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8400, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8560, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8460, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8500, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.847

In [91]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times1).mean(), torch.tensor(times1).std()))

Mean: 16.312, Std: 0.113


In [176]:
accuracies2 = []
times2 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxBaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.6)
    time2 = time.perf_counter()
    times2.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies2.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies2).mean(), torch.tensor(accuracies2).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1., device='cuda:0') tensor(0.8440, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8330, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8320, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8440, device='cuda:0')
tensor(0.9920, device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8490, device='cuda:0')
tensor(0.9980, device='cuda:0') tensor(0.8540, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8540, device='cuda:0')
tensor(0.9970, device='cuda:0') tensor(0.8820, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8410, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8260, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8430, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8280, device='cuda:0')
tensor(0.9880, devic

In [180]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times2).mean(), torch.tensor(times2).std()))

Mean: 23.097, Std: 0.146


In [185]:
accuracies3 = []
times3 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseBaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.7)
    time2 = time.perf_counter()
    times3.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies3.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies3).mean(), torch.tensor(accuracies3).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1., device='cuda:0') tensor(0.8710, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8730, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8920, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8700, device='cuda:0')
tensor(0.9620, device='cuda:0') tensor(0.8900, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8560, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8900, device='cuda:0')
tensor(0.9920, device='cuda:0') tensor(0.8950, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8710, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8680, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8450, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8830, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8720, device='cuda:0')
tensor(0.9920, device='cuda:0') tensor(0.8910, device='cuda:0')
tensor(0.9980, d

In [186]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times3).mean(), torch.tensor(times3).std()))

Mean: 18.741, Std: 0.118


In [167]:
accuracies4 = []
times4 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = ResNet(nb_residual_blocks = 4, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False)
    time2 = time.perf_counter()
    times4.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies4.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies4).mean(), torch.tensor(accuracies4).std()))

tensor(1., device='cuda:0') tensor(0.8130, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8380, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8140, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8420, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8210, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8250, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8410, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8340, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8530, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8290, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8370, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8230, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8480, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8410, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.858

In [254]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times4).mean(), torch.tensor(times4).std()))

Mean: 38.713, Std: 0.151


In [169]:
accuracies5 = []
times5 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    model = SiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    #model = BaseNet()
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True)
    time2 = time.perf_counter()
    times5.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_siamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies5.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies5).mean(), torch.tensor(accuracies5).std()))

tensor(1., device='cuda:0') tensor(0.8620, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8580, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8730, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8670, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8600, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8600, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8570, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8520, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8830, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8770, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8900, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8630, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8540, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8840, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8750, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8510, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.861

In [255]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times5).mean(), torch.tensor(times5).std()))

Mean: 53.883, Std: 0.104


In [89]:
accuracies6 = []
times6 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxResNet(nb_residual_blocks = 10, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.9)
    time2 = time.perf_counter()
    times6.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies6.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies6).mean(), torch.tensor(accuracies6).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.9870, device='cuda:0') tensor(0.8550, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8690, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8790, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8800, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8590, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9010, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8720, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(0.9950, device='cuda:0') tensor(0.8660, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8760, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8860, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8740, device='cuda:0')
tensor(0.9990, device='cuda:0') tensor(0.8920, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8590, device='cuda:0')
tensor(1., device='cuda:

In [256]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times6).mean(), torch.tensor(times6).std()))

Mean: 164.991, Std: 0.467


In [187]:
accuracies7 = []
times7 = []

for i in range(20):
    train_loader, test_loader = load_data(N=1000, batch_size=50, seed=i)
    time1 = time.perf_counter()
    #model = SiameseBaseNet()
    #model = BaseNet()
    model = AuxsiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
    model.to(device)
    train(model, train_loader, 0.001, 0, 25, verbose=False, siamese=True, aux=True, alpha = 0.7)
    time2 = time.perf_counter()
    times7.append(time2 - time1)

    tr_accuracy = 1 - compute_nb_errors_auxsiamese(model, train_loader)/1000
    te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
    print(tr_accuracy, te_accuracy)
    accuracies7.append(te_accuracy)
    
print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies7).mean(), torch.tensor(accuracies7).std()))

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(1., device='cuda:0') tensor(0.8870, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8870, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8780, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8800, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8830, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8790, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8850, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9090, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8820, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8700, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8770, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9000, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8790, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8940, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.9070, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.8830, device='cuda:0')
tensor(1., device='cuda:0') tensor(0.893

In [257]:
print('Mean: %.3f, Std: %.3f' %(torch.tensor(times7).mean(), torch.tensor(times7).std()))

Mean: 58.627, Std: 0.095


In [315]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies1 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = MLP()
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False)
            te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies1[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies1)

tensor([[0.8014, 0.8035, 0.8015, 0.8056, 0.8019],
        [0.8044, 0.8070, 0.8068, 0.8042, 0.8011],
        [0.8072, 0.8071, 0.8053, 0.8004, 0.7953],
        [0.7977, 0.7913, 0.7888, 0.7791, 0.7683]])


In [316]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies2 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = SiameseMLP()
            model.to(device)
            #model = BaseNet()
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=True)
            te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies2[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies2)

tensor([[0.8493, 0.8444, 0.8468, 0.8429, 0.8374],
        [0.8423, 0.8384, 0.8367, 0.8363, 0.8296],
        [0.8406, 0.8350, 0.8342, 0.8300, 0.8140],
        [0.8195, 0.8094, 0.7905, 0.7710, 0.7423]])


In [317]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies3 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = AuxMLP()
            model.to(device)
            #model = BaseNet()
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.0)
            te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies3[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies3)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.8184, 0.8225, 0.8223, 0.8176, 0.8120],
        [0.8139, 0.8119, 0.8114, 0.8063, 0.8106],
        [0.8130, 0.8087, 0.8095, 0.8112, 0.8017],
        [0.8060, 0.7966, 0.7876, 0.7730, 0.7571]])


In [319]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies5 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = BaseNet()
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False)
            te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies5[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies5)

tensor([[0.7246, 0.7686, 0.7808, 0.8034, 0.8205],
        [0.8241, 0.8299, 0.8253, 0.8273, 0.8256],
        [0.8287, 0.8311, 0.8293, 0.8291, 0.8189],
        [0.8303, 0.8192, 0.7826, 0.7837, 0.7460]])


In [320]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies6 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = SiameseBaseNet()
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=True)
            te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies6[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies6)

tensor([[0.8408, 0.8178, 0.8411, 0.8424, 0.8413],
        [0.8546, 0.8564, 0.8523, 0.8487, 0.8513],
        [0.8520, 0.8536, 0.8530, 0.8507, 0.8529],
        [0.8486, 0.8511, 0.8439, 0.8258, 0.7881]])


In [321]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies7 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = AuxBaseNet()
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.0)
            te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies7[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies7)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.7255, 0.7897, 0.8042, 0.7796, 0.8148],
        [0.8354, 0.8351, 0.8372, 0.8389, 0.8329],
        [0.8410, 0.8391, 0.8380, 0.8347, 0.8325],
        [0.8324, 0.8310, 0.8204, 0.8055, 0.7755]])


In [323]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies9 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = ResNet(nb_residual_blocks = 4, input_channels = 2, nb_channels = 32, kernel_size = 3, nb_classes = 2)
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False)
            te_accuracy = 1 - compute_nb_errors(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies9[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies9)

tensor([[0.8262, 0.8285, 0.8360, 0.8349, 0.8226],
        [0.8398, 0.8384, 0.8403, 0.8343, 0.8302],
        [0.8354, 0.8340, 0.8184, 0.8279, 0.8218],
        [0.8152, 0.8161, 0.8157, 0.8150, 0.8147]])


In [324]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies10 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = SiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=True)
            te_accuracy = 1 - compute_nb_errors_siamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies10[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies10)

tensor([[0.8643, 0.8631, 0.8770, 0.8667, 0.8742],
        [0.8735, 0.8755, 0.8733, 0.8703, 0.8643],
        [0.8751, 0.8685, 0.8687, 0.8601, 0.8569],
        [0.8529, 0.8498, 0.8487, 0.8552, 0.8570]])


In [325]:
gammas = [5e-3, 1e-3, 5e-4, 1e-4]
batch_sizes = [8, 16, 32, 64, 128]
test_accuracies11 = torch.empty((len(gammas), len(batch_sizes)))

for j in range(len(gammas)):
    for k in range(len(batch_sizes)):
        accurate = []
        for i in range(10):
            model = AuxResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
            model.to(device)
            train_loader, test_loader = load_data(N=1000, batch_size=batch_sizes[k], seed=i)
            train(model, train_loader, gammas[j], 0, 25, verbose=False, siamese=False, aux=True, alpha = 0.0)
            te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
            accurate.append(te_accuracy)
        test_accuracies11[j,k] =  torch.cuda.FloatTensor(accurate).mean()
print(test_accuracies11)

C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Users\laboleb\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.8342, 0.8438, 0.8438, 0.8465, 0.8444],
        [0.8453, 0.8489, 0.8476, 0.8429, 0.8363],
        [0.8432, 0.8423, 0.8421, 0.8375, 0.8292],
        [0.8244, 0.8290, 0.8252, 0.8305, 0.8376]])


In [ ]:
for j in range(11):
    accuracies100 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=16, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxMLP()()
        train(model, train_loader, 5e-3, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies100.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies100).mean(), torch.tensor(accuracies100).std()))

In [ ]:
for j in range(11):
    accuracies101 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=8, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseMLP()
        train(model, train_loader, 5e-3, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies101.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies101).mean(), torch.tensor(accuracies101).std()))

In [ ]:
for j in range(11):
    accuracies102 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=16, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseBaseNet()
        train(model, train_loader, 1e-3, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies102.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies102).mean(), torch.tensor(accuracies102).std()))

In [ ]:
for j in range(11):
    accuracies103 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=8, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxBaseNet()
        train(model, train_loader, 5e-4, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies103.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies103).mean(), torch.tensor(accuracies103).std()))

In [ ]:
for j in range(11):
    accuracies104 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=16, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseResNet(nb_residual_blocks = 4, input_channels = 1, nb_channels = 32, kernel_size = 3, nb_classes = 2)
        train(model, train_loader, 1e-3, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies104.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies104).mean(), torch.tensor(accuracies104).std()))

In [ ]:
for j in range(11):
    accuracies105 = []

    for i in range(10):
        train_loader, test_loader = load_data(N=1000, batch_size=8, seed=i)

        #model = SiameseBaseNet()
        #model = BaseNet()
        model = AuxsiameseMLP()
        train(model, train_loader, 5e-3, 0, 25, verbose=False, siamese=False, aux=True, alpha = j/10)

        te_accuracy = 1 - compute_nb_errors_auxsiamese(model, test_loader)/1000
        accuracies105.append(te_accuracy)
    print('Mean: %.3f, Std: %.3f' %(torch.tensor(accuracies105).mean(), torch.tensor(accuracies105).std()))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same